<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Ответим-на-вопросы-по-задаче-исследования" data-toc-modified-id="Ответим-на-вопросы-по-задаче-исследования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ответим на вопросы по задаче исследования</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Задача от кредитного отдела банка.

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов. 

Результаты исследования будут использованы при построении модели кредитного скоринга(системы, которая оценивает способность потенциального заёмщика вернуть кредит банку)


##  Подготовка

In [88]:
#  Откроем файл с данными и изучите общую информацию
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

В данных есть нужные метрики, подходящие для проверки гипотезы.
В данных есть мусор (пропуски, дубликаты и ошибочно отрицательные значения).

---

## Предобработка данных

### Обработка пропусков

In [89]:
#посмотрим сколько всего пропусков по колонкам
data.isnull().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропуски есть в колонках days_employed и total_income.
Колонка days_employed нам не нужна для проверки гипотез, так что в ней пропуски можно не заменять.


In [90]:
# для экономии места отбросим знаки до 1 после запятой и заодно избавимся от ложно отрицательных значений
# в колонке days_employed
data['days_employed'] = abs(data['days_employed'])
data['days_employed'] = round(data['days_employed'], 1)

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.7,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.8,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.4,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.7,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.1,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [91]:
# чтобы понять, чем заменять пропуски, посмотрим насколько большой разброс значений в столбце total_income
# найдем максимальное значение
data['total_income'].max()

2265604.028722744

In [92]:
# найдем минимальное значение
data['total_income'].min() 

20667.26379327158

Сравнили минимальное и максимальное значения.
Разница больше чем в 10 раз, поэтому будем заменять медианой.

In [93]:
# найдем медианы по разным группам income_type
data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [94]:
# заменим пропуски для каждой категории income_type соответствующей медианой
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.7,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.8,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.4,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.7,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.1,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.2,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.8,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.9,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.8,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [95]:
# проверим снова наличие пропусков
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

**Вывод**

Пропуски в колонке с общим доходом синхронны с пропусками в колонке стажа.
Для проверки гипотез нам не понадобятся данные стажа, так что пропуски заменены только в данных общего дохода.
Из за довольно большого разброса значений, пропуски заменили медианой. 
Медиану посчитали для каждого типа занятости(пенсионеры/сотрудники/госслужащие и тд), чтобы уменьшить погрешность.

---

### Замена типа данных

In [96]:
data.info() #смотрим какие типы данных содержатся

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [97]:
# заменим данные в столбце total_income на целочисленные, методом astype()
data['total_income'] = data['total_income'].astype('int')
data.head(15)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

Нам не обязательно знать доход вплоть до миллионной доли рубля, так что для удобства данные в столбце общего дохода заменили на целочисленные.

---

### Обработка дубликатов

In [98]:
# приводим данные к нижнему регистру методом str.lower() в колонках, где есть текст
data['education'] = data['education'].str.lower() 
data['family_status'] = data['family_status'].str.lower() 
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.7,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья
1,1,4024.8,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623.4,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья
3,3,4124.7,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование
4,0,340266.1,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу
5,0,926.2,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья
6,0,2879.2,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем
7,0,152.8,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование
8,2,6929.9,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы
9,0,2188.8,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи


In [99]:
# удалим все дубликаты методом drop_duplicates(), обновляя индексы
data = data.drop_duplicates().reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


In [100]:
# чтобы проверить на ошибки, рассмотрим уникальные значения данных для каждого столбца
for row in data: 
  print(data[row].value_counts()) 

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
204.2       5
865.0       5
935.6       5
929.1       5
1481.6      5
           ..
386310.9    1
1264.8      1
716.2       1
3681.8      1
1078.1      1
Name: days_employed, Length: 16990, dtype: int64
35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
37    536
48    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
59    443
55    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64    260
23    252
65    193
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее                15172
высшее                  5

In [101]:
#заменим значения "20" и "-1" в столбце children
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1
for row in data: 
  print(data[row].value_counts())


0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64
204.2       5
865.0       5
935.6       5
929.1       5
1481.6      5
           ..
386310.9    1
1264.8      1
716.2       1
3681.8      1
1078.1      1
Name: days_employed, Length: 16990, dtype: int64
35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
37    536
48    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
59    443
55    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64    260
23    252
65    193
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее                15172
высшее                  5250
неоконченное высшее      744
начал

**Вывод**

Удалили дубликаты, количество строчек сократилось с 21525 до 21454.
В колонке количества детей исправили ошибочные значения "20" и "-1", совершенные скорее всего человеком в момент заполнения этой графы.

---

### Лемматизация

In [102]:
# импортируем библиотеку
from pymystem3 import Mystem
m = Mystem()

# создаем столбец, куда созраняются леммы для каждой строчки
def purpose_lemmas(row):
    lemmas = m.lemmatize(row['purpose'])
    return ''.join(lemmas)
data['purpose_lemmas'] = data.apply(purpose_lemmas, axis =1)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,8437.7,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,покупка жилье\n
1,1,4024.8,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n
2,0,5623.4,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,покупка жилье\n
3,3,4124.7,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,дополнительный образование\n
4,0,340266.1,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n


In [103]:
# выведем полный список лемм для колонки purpose
from collections import Counter
Counter(data['purpose_lemmas'])

Counter({'покупка жилье\n': 646,
         'приобретение автомобиль\n': 461,
         'дополнительный образование\n': 460,
         'сыграть свадьба\n': 765,
         'операция с жилье\n': 652,
         'образование\n': 447,
         'на проведение свадьба\n': 768,
         'покупка жилье для семья\n': 638,
         'покупка недвижимость\n': 621,
         'покупка коммерческий недвижимость\n': 661,
         'покупка жилой недвижимость\n': 606,
         'строительство собственный недвижимость\n': 635,
         'недвижимость\n': 633,
         'строительство недвижимость\n': 619,
         'на покупка подержать автомобиль\n': 478,
         'на покупка свой автомобиль\n': 505,
         'операция с коммерческий недвижимость\n': 650,
         'строительство жилой недвижимость\n': 624,
         'жилье\n': 646,
         'операция со свой недвижимость\n': 627,
         'автомобиль\n': 972,
         'заниматься образование\n': 408,
         'сделка с подержанный автомобиль\n': 486,
         'получ

**Вывод**

Создана колонка, содержащая все лемматизированные цели взятия кредитов.

---

### Категоризация данных

In [104]:
# просматриваем список всех лемм и вручную выделяем несколько категорий
# создаем функцию, которая будет принимать на входе лемматизировнную цель кредита и выдавать категорию,
# к которой эта цель относится

def purpose_group(row):
    lem=row['purpose_lemmas']
    if 'недвижимость' in lem or 'жилье' in lem:
        return 'недвижимость'
    elif 'образование' in lem:
        return 'образование'
    elif 'свадьба' in lem: 
        return 'свадьба'
    elif 'автомобиль' in lem:
        return 'автомобиль'
    else:
        return "прочее"
    
data['purpose_lemmas'] = data.apply(purpose_group, axis=1)
data.head(10)
data['purpose_lemmas'].value_counts().head()


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_lemmas, dtype: int64

**Вывод**

Все цели взятия кредита можно поделить на 4 категории: недвижимость, автомобиль, образование и свадьба

---

## Ответим на вопросы по задаче исследования

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [105]:
# создадим сводную таблицу по количеству детей
# и колонками sum (количество должников) и count (все люди, взявшие кредит)
child_data = data.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum', 'count']).reset_index()      
child_data

,children,sum,count
,,debt,debt
0,0,1063,14091
1,1,445,4855
2,2,202,2128
3,3,27,330
4,4,4,41
5,5,0,9


In [106]:
# добавим колонку с процентом должников
child_data['debts_by_children, %'] = child_data['sum'] / child_data['count'] *100
child_data

,children,sum,count,"debts_by_children, %"
,,debt,debt,
0,0,1063,14091,7.543822
1,1,445,4855,9.165808
2,2,202,2128,9.492481
3,3,27,330,8.181818
4,4,4,41,9.756098
5,5,0,9,0.000000


In [107]:
# посчитаем общий процент должников для всех людей, у кого есть хотя бы один ребенок
children_data_debt = child_data[child_data['children'] > 0]['sum'].sum()
children_data_total = child_data[child_data['children'] > 0]['count'].sum()
children_data = children_data_debt / children_data_total * 100
children_data

debt    9.208203
dtype: float64

**Вывод**

Процент должников среди тех, у кого нет детей - 7.5 %
Процент должников среди тех, у кого есть хотя бы один ребенок - 9.2 %

Соответственно, люди без детей чаще возвращают кредиты в срок, нежели люди, у которых есть хотя бы один ребенок.


---

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [108]:
# создадим сводную таблицу по семейному положению и колонками sum (количество должников) 
# и count (все люди, взявшие кредит)
family_data = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum', 'count']).reset_index()
family_data

,family_status,sum,count
,,debt,debt
0,в разводе,85,1195
1,вдовец / вдова,63,959
2,гражданский брак,388,4151
3,женат / замужем,931,12339
4,не женат / не замужем,274,2810


In [109]:
# добавим колонку с процентом должников
family_data['debts_by_family_status, %'] = family_data['sum'] / family_data['count'] *100
family_data

,family_status,sum,count,"debts_by_family_status, %"
,,debt,debt,
0,в разводе,85,1195,7.112971
1,вдовец / вдова,63,959,6.569343
2,гражданский брак,388,4151,9.347145
3,женат / замужем,931,12339,7.545182
4,не женат / не замужем,274,2810,9.750890


**Вывод**

Неженатые/незамужние и состоящие в гражданском браке люди чаще всего являются должниками по кредиту - 9.75% и 9.35%  должников соответственно.
Женатые/замужние и люди в разводе немного чаще возвращают кредиты - 7.55% и 7.11%  должников

Реже всего должниками являются вдовцы и вдовы - 6.57% должников


---

- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [110]:
# для разделения на уровни дохода решила воспользоваться квантилями
# находим квантиль, до которого будет низкий доход
data['total_income'].quantile(0.25) 

107623.0

In [111]:
# находим квантиль, после которого будет высокий доход
# средний доход между ними
data['total_income'].quantile(0.75)

195820.25

In [112]:
# делим данные в колонке total_income на низкий, средний и высокий уровни дохода

def income_level(row):
    if row['total_income'] <= 107623.0 :
        return 'низкий'
    elif 107623.0  < row['total_income'] < 195820.25:
        return 'средний'
    else:
        return 'высокий'
data['total_income_status'] = data.apply(income_level, axis =1)
data.head(10)
# смотрим общее количество человек по каждому уровню дохода
data['total_income_status'].value_counts()

средний    10726
высокий     5364
низкий      5364
Name: total_income_status, dtype: int64

In [113]:
# создадим сводную таблицу по уровню дохода и колонками sum (количество должников) 
# и count (все люди, взявшие кредит)
total_income_data = data.pivot_table(index = ['total_income_status'], values = 'debt', aggfunc = ['sum', 'count']).reset_index()
total_income_data

,total_income_status,sum,count
,,debt,debt
0,высокий,383,5364
1,низкий,427,5364
2,средний,931,10726


In [114]:
# добавим колонку с процентом должников
total_income_data['debts_by_total_income, %'] = total_income_data['sum'] / total_income_data['count'] *100
total_income_data

,total_income_status,sum,count,"debts_by_total_income, %"
,,debt,debt,
0,высокий,383,5364,7.140194
1,низкий,427,5364,7.960477
2,средний,931,10726,8.679843


**Вывод**

Люди с высоким уровнем дохода реже всего оказываются должниками по кредиту - 7.14% должников

Люди с низким уровнем дохода немного чаще оказываются должниками - 7.96% должников

Чаще всего должниками оказываются люди со средним уровнем дохода - 8.68% должников

---

- **Как разные цели кредита влияют на его возврат в срок?**

In [115]:
# создадим сводную таблицу по цели кредита и колонками sum (количество должников) 
# и count (все люди, взявшие кредит)
purpose_data = data.pivot_table(index=['purpose_lemmas'], values='debt', aggfunc = ['sum', 'count']).reset_index()
purpose_data

,purpose_lemmas,sum,count
,,debt,debt
0,автомобиль,403,4306
1,недвижимость,782,10811
2,образование,370,4013
3,свадьба,186,2324


In [116]:
# добавим колонку с процентом должников
purpose_data['debts_by_purpose, %'] = purpose_data['sum'] / purpose_data['count'] *100
purpose_data

,purpose_lemmas,sum,count,"debts_by_purpose, %"
,,debt,debt,
0,автомобиль,403,4306,9.359034
1,недвижимость,782,10811,7.233373
2,образование,370,4013,9.220035
3,свадьба,186,2324,8.003442


**Вывод**


Чаще всего возвращают кредит, взятый на операции с недвижимостью - 7.23% должников

Немного реже возвращают кредиты, взятые на свадьбу - 8% должников

Реже всего возвращается кредит, взятый на автомобиль и образование - 9.36% и 9.22% должников


---

## Общий вывод

- Наличие детей скорее негативно отражается на количестве должников по кредиту. Бездетные чаще возвращают кредиты в срок, нежели люди, у которых есть хотя бы один ребенок.


- Неженатые/незамужние и состоящие в гражданском браке люди чаще всего являются должниками по кредиту. Женатые/замужние и люди в разводе немного чаще возвращают кредиты.
Реже всего должниками являются вдовцы и вдовы.


- Чаще всего должниками оказываются люди со средним уровнем дохода.
Люди с низким уровнем дохода немного чаще оказываются должниками.
Люди с высоким уровнем дохода реже всего оказываются должниками по кредиту.


- Все цели взятия кредита можно поделить на 4 категории: недвижимость, автомобиль, образование и свадьба.
Чаще всего возвращают кредит, взятый на операции с недвижимостью и свадьбы.
Реже всего возвращается кредит, взятый на автомобиль и образование.
